### Custom FaceSwap from scratch with dlib and opencv

In [1]:
# Import the required libraries
import cv2
import numpy as np
import dlib

In [2]:
#initialize dlib library's face detector
#create dlib library's facial landmark predictor
frontal_face_detector = dlib.get_frontal_face_detector()
frontal_face_predictor = dlib.shape_predictor("dataset/shape_predictor_68_face_landmarks.dat")

#read the source face image and convert it to grayscale
source_image = cv2.imread("images/ayobami.jpg")
source_image_grayscale = cv2.cvtColor(source_image, cv2.COLOR_BGR2GRAY)
#cv2.imshow("source_image",source_image)

#read the destination face image and convert it to grayscale
destination_image = cv2.imread("images/ayodeji.jpg")
destination_image_grayscale = cv2.cvtColor(destination_image, cv2.COLOR_BGR2GRAY)
#cv2.imshow("destination_image",destination_image)

#create a zeros array canvas exactly like the same size of source_image_grayscale
source_image_canvas = np.zeros_like(source_image_grayscale)

#getting the shape of destination_image
height, width, no_of_channels = destination_image.shape

#create a zeros array canvas like the destination image
destination_image_canvas = np.zeros((height,width,no_of_channels),np.uint8)

#define a method to get the index
def index_from_array(numpyarray):
    index = None
    for n in numpyarray[0]:
        index = n
        break
    return index


#FOR THE SOURCE IMAGE
####################
#Find the faces in source image
#Returns a numpy array containing a histogram of pixels in the image
source_faces = frontal_face_detector(source_image_grayscale)

#Loop through all faces found in the source image
for source_face in source_faces:
    source_face_landmarks = frontal_face_predictor(source_image_grayscale, source_face)
    source_face_landmark_points = []
    
    # Loop through all 68 landmark points
    for landmark_no in range(0, 68):
        x_point = source_face_landmarks.part(landmark_no).x
        y_point = source_face_landmarks.part(landmark_no).y
        source_face_landmark_points.append((x_point, y_point))
        
#         cv2.circle(source_image, (x_point, y_point), 2, (255, 0, 0), -1)
# cv2.imshow('Landmarks', source_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
print(source_face_landmark_points)

[(320, 324), (322, 370), (324, 414), (329, 460), (343, 507), (366, 547), (398, 585), (442, 614), (496, 624), (550, 621), (594, 597), (627, 566), (651, 528), (663, 483), (669, 435), (675, 390), (681, 343), (349, 284), (374, 260), (410, 253), (446, 262), (479, 280), (535, 280), (569, 264), (605, 258), (640, 269), (662, 296), (507, 317), (507, 344), (506, 371), (506, 400), (463, 420), (482, 427), (505, 434), (527, 429), (547, 423), (387, 321), (408, 311), (433, 313), (453, 329), (430, 332), (406, 331), (559, 333), (582, 318), (607, 317), (625, 330), (607, 338), (583, 337), (422, 478), (452, 467), (483, 463), (504, 469), (525, 465), (556, 472), (585, 488), (557, 511), (526, 518), (503, 519), (480, 516), (449, 503), (434, 481), (483, 485), (504, 488), (525, 487), (574, 491), (524, 490), (503, 491), (482, 487)]


In [4]:
print(source_faces)

rectangles[[(286, 187) (732, 633)]]
